## Course Assignment Instructions
You should have Python (version 3.8 or later) and Jupyter Notebook installed to complete this assignment. You will write code in the empty cell/cells below the problem. While most of this will be a programming assignment, some questions will ask you to "write a few sentences" in markdown cells. 

Submission Instructions:

Create a labs directory in your personal class repository (e.g., located in your home directory)
Clone the class repository
Copy this Jupyter notebook file (.ipynb) into your repo/labs directory
Make your edits, commit changes, and push to your repository
All submissions must be pushed before the due date to avoid late penalties. 

Labs are graded out of a 100 pts. Each day late is -10. For a max penalty of -50 after 5 days. From there you may submit the lab anytime before the semester ends for a max score of 50.  

Lab 4 is due on 3/3/25 

Create a dataset D which we call `Xy` such that the linear model has  R² about 0\% but x, y are clearly associated.

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from plotnine import ggplot, aes, geom_point, scale_y_continuous, labs, ggtitle

# Set numpy to ignore overflow warnings
np.seterr(over='ignore')

# Create x values from 1 to 200
x = np.arange(1, 201, dtype=np.float64)

# Compute y as x raised to the 67th power
y = x ** 67

# Create the DataFrame
df = pd.DataFrame({'x': x, 'y': y})

# Fit the linear model: y ~ x
X = sm.add_constant(df['x'])  # Adds an intercept
model = sm.OLS(df['y'], X).fit()
print("R-squared:", model.rsquared)

# Plot using plotnine with specified y-axis limits
p = (
    ggplot(df, aes(x='x', y='y'))
    + geom_point()
    + scale_y_continuous(limits=(-7.37e152, 20))
    + labs(x='x', y='y')
    + ggtitle('Scatter Plot of x vs y')
)

p


The issue is that when you compute y = x⁶⁷ on x values from 1 to 200, the resulting numbers are extremely large. Even when computed as floats, this can lead to numerical instability in the regression calculation, causing R² to become NaN. One common fix is to scale down y before fitting the model. Run the code below and see the difference. 

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from plotnine import ggplot, aes, geom_point, scale_y_continuous, labs, ggtitle

# Set numpy to ignore overflow warnings during exponentiation
np.seterr(over='ignore')

# Create x values as floats
x = np.arange(1, 201, dtype=np.float64)

# Compute y as x raised to the 67th power
y = x ** 67

# Scale down y for regression purposes (R² is invariant under scaling)
scaling_factor = 1e154
y_scaled = y / scaling_factor

# Build the DataFrame with original and scaled y values
df = pd.DataFrame({'x': x, 'y': y, 'y_scaled': y_scaled})

# Fit the linear model on the scaled y
X = sm.add_constant(df['x'])
model = sm.OLS(df['y_scaled'], X).fit()
print("R-squared:", model.rsquared)

# Define the y-axis limits as Python floats
y_min = float(-7.37e152)
y_max = float(1.54e154)

# Provide explicit breaks and labels to avoid automatic tick calculation issues
p = (
    ggplot(df, aes(x='x', y='y'))
    + geom_point()
    + scale_y_continuous(
          limits=(y_min, y_max),
          breaks=[y_min, 0.0, y_max],
          labels=[f"{y_min:.2e}", "0", f"{y_max:.2e}"]
      )
    + labs(x='x', y='y')
    + ggtitle('Scatter Plot of x vs y')
)

print(p)
p

Write a function `my_ols` that takes in `X`, a matrix with with p columns representing the feature measurements for each of the n units, a vector of n responses `y` and returns a list that contains the `b`, the p+1-sized column vector of OLS coefficients, `yhat` (the vector of n predictions), `e` (the vector of n residuals), `df` for degrees of freedom of the model, `SSE`, `SST`, `MSE`, `RMSE` and `Rsq` (for the  R² metric). You will have to use `linalg.inv` from numpy

In [ ]:
import numpy as np
import pandas as pd

# Define a custom class to mimic R's class assignment
class MyOLSResult(dict):
    pass

def my_ols(X, y, add_intercept=True):
    # Convert inputs to numpy arrays
    X = np.array(X)
    y = np.array(y)
    
    # Check that X is numeric
    if not np.issubdtype(X.dtype, np.number):
        raise ValueError("X is not numeric")
    # Check that y is numeric
    if not np.issubdtype(y.dtype, np.number):
        raise ValueError("y needs to be numeric")
    # Check that the number of rows in X equals the length of y
    if X.shape[0] != len(y):
        raise ValueError("X rows and length of y need to be the same length.")
    
    n = len(y)
    
    # Add an intercept column if requested
    if add_intercept:
        X = np.column_stack((np.ones(n), X))
    
    p = X.shape[1]  # p is the number of parameters (including the intercept if added)
    df = n - p      # residual degrees of freedom
    
    if n <= p:
        raise ValueError("There must be more observations than parameters")
    
    y_bar = np.mean(y)
    
    # Compute OLS coefficients: b = (X^T X)^{-1} X^T y
    XtX = X.T @ X
    b = np.linalg.inv(XtX) @ (X.T @ y)
    
    # Compute predictions and residuals
    yhat = X @ b
    e = y - yhat
    
    # Compute SSE and SST
    SSE = e.T @ e
    SST = np.sum((y - y_bar)**2)
    
    # Compute MSE, RMSE, and R-squared
    MSE = SSE / (n - p)
    RMSE = np.sqrt(MSE)
    Rsq = 1 - (SSE / SST)
    
    # Construct the result as an instance of MyOLSResult
    result = MyOLSResult({
        'b': b,
        'yhat': yhat,
        'df': df,
        'e': e,
        'SSE': SSE,
        'SST': SST,
        'MSE': MSE,
        'RMSE': RMSE,
        'Rsq': Rsq,
        'p': p
    })
    return result


Verify that the OLS coefficients for the `Type` of cars in the cars dataset gives you the same results as we did in class (i.e. the ybar's within group). You will load the cars data set by importing data from pydataset 

In [ ]:
from pydataset import data

# Load the Cars93 dataset (equivalent to MASS::Cars93)


# Create a design matrix for Type with no intercept,
# similar to model.matrix(~ 0 + Type, data = cars) in R.


# Get the Price vector


# Run our custom OLS function with add_intercept set to False
result = my_ols()
print(result)

Create a prediction method `g` that takes in a vector `x_star` and the dataset D i.e. `X` and `y` and returns the OLS predictions. Let `X` be a matrix with with p columns representing the feature measurements for each of the n units


In [ ]:
import numpy as np

def g(x_star, X, y, add_intercept=True):
    """
    Prediction function that returns the OLS prediction for a new observation.
    
    Parameters:
        x_star : array-like
            A vector of feature values for the new observation.
        X : array-like
            The design matrix for the training data.
        y : array-like
            The response vector for the training data.
        add_intercept : bool (default True)
            Whether the my_ols function should add an intercept.
    
    Returns:
        The OLS prediction for x_star.
    """
    # Get the OLS result from our custom function
    
    
    # Ensure x_star is a numpy array
 
    
    # If an intercept was added in the model, prepend a 1 to x_star.
    
    # Return the dot product (prediction)
    

# Create a design matrix for Type with no intercept, and force numeric conversion
X = pd.get_dummies(cars['Type'], prefix='Type', drop_first=False).astype(float)
y = cars['Price'].to_numpy()

# Convert X to a NumPy array so that indexing works as expected
X_np = X.to_numpy()

# Now, predict for the first observation in X using our custom function g
prediction = g()
print(prediction)

Load up the famous iris dataset. We are going to do a different prediction problem. Imagine the only input x is Species and you are trying to predict y which is Petal.Length. A reasonable prediction is the average petal length within each Species. Prove that this is the OLS model by fitting an appropriate `ols` and then using the `mod.predict` function to verify. Note: this time we will load the iris dataset from another python package called seaborn which we will cover in dept during lab 6 when we go over visualizations. 

In [ ]:
#pip install seaborn

In [ ]:
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf

# Load the iris dataset from seaborn
iris = sns.load_dataset("iris")

# Optionally, rename columns to match the R names (Petal.Length becomes Petal_Length and Species stays Species)
iris.rename(columns={"petal_length": "Petal_Length", "species": "Species"}, inplace=True)

# Fit the OLS model: Petal_Length ~ Species.
# This will create dummy variables internally and use one species as the baseline.
mod = smf.ols().fit()

# Compute the mean Petal_Length within each Species
mean_setosa = iris.loc[iris["Species"] == "setosa", "Petal_Length"].mean()


print("Mean Petal_Length for setosa: ", mean_setosa)


# Use predict to obtain predictions for each Species.
pred_setosa = mod.predict(pd.DataFrame({"Species": ["setosa"]}))


print("Predicted Petal_Length for setosa: ", pred_setosa.iloc[0])



You may have noticed that to get the OLS function from statsmodel we used smf instead of sm. Either method can be used but the
difference is in the interface they provide:

sm (statsmodels.api):
This is the lower-level API where you work directly with data arrays or matrices. You have to manually create the design matrix (including adding an intercept if needed), and then pass it to functions like sm.OLS. This interface is similar to using raw matrix algebra.

smf (statsmodels.formula.api):
This offers a higher-level, formula-based interface (similar to R's lm), where you can specify your model using a formula string (e.g., "y ~ x") and a DataFrame. The function automatically constructs the design matrix and handles categorical variables.

Construct the design matrix with an intercept, X manually. You will need to use np.column_stack

In [ ]:
# Construct the design matrix manually:
# - The first column is an intercept (all ones)
# - The second column is 1 if species is "versicolor", else 0
# - The third column is 1 if species is "virginica", else 0
X = np.column_stack((
    np.ones(),
    (iris['Species'] == "versicolor").astype(int),
    ()
))

# Show the first few rows of the design matrix
print()

We now import the diamonds dataset from plotnine.data. In R you Skim the dataset using skimr or summary. Here we will use .info() and/or .describe from pandas. What is the datatype of the color feature? (hint: use .dtype on the color feature from the dataframe.)

In [ ]:
import pandas as pd
from plotnine.data import diamonds


Find the levels of the color feature. Using pandas this can be accomplished by assigning the feature "color" to a variable by using .cat.categories from pandas.

In [ ]:
# Get the levels (categories) of the 'color' feature.


Create new feature in the diamonds dataset, `color_as_numeric`, which is color expressed as a continuous interval value. Use .cat.codes from pandas.

In [ ]:
# Create a new feature 'color_as_numeric' with 1-based coding (like R)


Use that converted feature as the one predictor in a regression. How well does this regression do as measured by RMSE? Use sm from statsmodels.api

In [ ]:
import statsmodels.api as sm


Try this again using smf from statsmodels.api and see if the result matches the previous cell

In [ ]:
import statsmodels.formula.api as smf


Create new feature in the diamonds dataset, `color_as_nominal`, which is color expressed as a nominal categorical variable. Use .Categorial and set ordered = False

In [ ]:
# Create a new feature 'color_as_nominal' as a nominal (unordered) categorical variable


#Creating the Nominal Feature:
#We use pd.Categorical to convert the "color" column into a categorical variable. 
#By setting ordered=False, we ensure it is treated as nominal (unordered).

Use that converted feature as the one predictor in a regression. How well does this regression do as measured by RMSE?

Which regression does better - `color_as_numeric` or `color_as_nominal`? Why?

Now regress both `color_as_numeric` and `color_as_nominal` in a regression. Does this regression do any better (as gauged by RMSE) than either color_as_numeric` or `color_as_nominal` alone?

In [ ]:
# Model using both predictors


What are the coefficients (the b vector)? You can access the parameters using .params and assigning it to a variable `b` 

In [ ]:
# Extract the coefficient vector (b vector)


Your Python results should be very similar to what you'd expect in R. The slight differences can be attributed to a few factors:

Factor Level Ordering:
R and Python (via pandas) may assign the underlying integer codes to factor levels in slightly different orders depending on the default level ordering. This can affect the dummy coding and, as a result, the estimated coefficients.

Numerical Precision and Implementation:
The linear algebra routines in R and Python (via NumPy/statsmodels) may differ slightly in numerical precision or the way they handle floating‐point arithmetic. These differences can lead to small discrepancies in coefficient estimates.

Contrast Coding Differences:
R’s default dummy coding (treatment contrasts) and statsmodels’ handling of categorical variables (which also uses treatment coding by default) are conceptually similar but might differ in subtle details (e.g., which level is chosen as the baseline).

Overall, the estimates are close enough that these differences are within the range expected from using different software implementations for the same model.

Something appears to be anomalous in the coefficients. What is it? Why?

Return to the iris dataset. Find the hat matrix H for this regression.

In [ ]:
from numpy.linalg import inv, matrix_rank

# Construct the design matrix X:
# 1 for the intercept, then indicators for Species == "versicolor" and Species == "virginica"
X = np.column_stack((
   
))

# Compute the hat matrix H = X (X^T X)^{-1} X^T
H = 

# Compute the rank of H
rank_H = matrix_rank()
print("Rank of H:", )

Verify this hat matrix is symmetric using the `assert_allclose` from `npt` from numpy.testing.

In [ ]:
try:
    
    print("Hat matrix is symmetric.")
except AssertionError as e:
    print("Hat matrix is not symmetric:", e)

Verify this hat matrix is idempotent using the `assert_allclose` from `npt` from numpy.testing.

In [ ]:
try:
   
    print("Hat matrix .")
except AssertionError as e:
    print("Hat matrix :", e)

Using the `np.diag` function from numpy, find the trace of the hat matrix (i.e. the sum of the diagonal elements in the matrix). 

In [ ]:
# Given H is your hat matrix

print("Trace of H:", )

It turns out the trace of a hat matrix is the same as its rank! But we don't have time to prove these interesting and useful facts..

Extra Credit (+5): create a matrix X-perpendicular

In [ ]:
# Assume H and X have been defined previously.


Using the hat matrix, compute the yhat vector and using the projection onto the residual space, compute the e vector and verify they are orthogonal to each other.

In [ ]:
# Get the response vector: equivalent to iris$Petal.Length in R


Compute SST, SSR and SSE and R² and then show that SST = SSR + SSE.

In [ ]:
# Compute SSE: sum of squared errors
SSE = 

# Compute y_bar: mean of y
y_bar = 

# Compute SST: total sum of squares
SST = 

# Compute SSR: regression sum of squares
SSR = 

print("SSR:", )
print("SSE:", )
print("SST:", )

# Verify that SST equals SSR + SSE within a tolerance

print("Verified: SST equals SSR + SSE.")

# Compute R² as SSR / SST
R_squared = SSR / SST
print("R²:", R_squared)

Find the angle theta between y - ybar 1 and yhat - ybar 1 and then verify that its cosine squared is the same as the R² from the previous problem.

In [ ]:
# Compute the dot product of (y - y_bar) and (y_hat - y_bar)

# Compute the denominator: sqrt(SST * SSR)


# Compute theta in radians using arccos


# Convert theta to degrees


print("Theta (in radians):", )
print("Theta (in degrees):", )

# Verify that cos^2(theta) is equal to R²

print("cos^2(theta):", )
print("R²:", )

Project the y vector onto each column of the X matrix and test if the sum of these projections is the same as yhat. Use `assert_allclose` from numpy (This should fail)

In [ ]:
# Projection onto the first column of X


Construct the design matrix without an intercept, X, without using pandas DataFrame.

In [ ]:
# Construct the design matrix without an intercept:
# Each column is an indicator (dummy variable) for one species.
X = pd.DataFrame({
    
})

# The response vector is petal_length.


# Show the first few rows of the design matrix


Find the OLS estimates using this design matrix. It should be the sample averages of the petal lengths within species.

Verify the hat matrix constructed from this design matrix is the same as the hat matrix constructed from the design matrix with the intercept by using `assert_allclose`. (Fact: orthogonal projection matrices are unique).

In [ ]:
# Verify that the two hat matrices are equal within numerical tolerance.


Project the y vector onto each column of the X matrix and test if the sum of these projections is the same as yhat.

In [ ]:
# Assuming H2 and y have already been computed (from previous steps),
# and yhatnew was computed as H2 @ y.


# Verify that the projection of y via H2 equals yhatnew


Convert this design matrix into Q, an orthonormal matrix.

In [ ]:
# Compute the QR decomposition of x. Q is the orthonormal matrix.
Q, R = np.linalg.qr(x) #tuple unpacking to assign Q (the orthonormal matrix) and R(the upper triangular matrix)

# Check normalization: Each column of Q should have unit length.
print("Sum of squares of Q[:,0]:", np.sum(Q[:, 0]**2))
print("Sum of squares of Q[:,1]:", np.sum(Q[:, 1]**2))


# Check orthogonality: Dot products between different columns should be 0.
print("Dot product Q[:,0] and Q[:,1]:", np.dot(Q[:, 0], Q[:, 1]))


Project the y vector onto each column of the Q matrix and test if the sum of these projections is the same as yhat.

In [ ]:
# Compute the projection onto each column of Q.
# Since Q is orthonormal, Q[:,i].T @ Q[:,i] == 1, so:
proj1 = (Q[:, 0:1] @ Q[:, 0:1].T) @ y


# Sum the individual projections.
yhatQ = 

# Alternatively, since Q is orthonormal, we can compute:
yhatQ_direct = Q @ Q.T @ y

# Verify that yhatQ equals yhatnew.
npt.assert_allclose(yhatQ, yhatnew, rtol=1e-7, atol=1e-10)
npt.assert_allclose(yhatQ_direct, yhatnew, rtol=1e-7, atol=1e-10)
print("The projection using Q equals yhatnew.")

Find the p=3 linear OLS estimates if Q is used as the design matrix using the `sm.OLS` from statsmodels. Is the OLS solution the same as the OLS solution for X?

In [ ]:
# Construct the design matrix x without an intercept:
# Each column is an indicator for one species.
x = np.column_stack((
    (iris['Species'] == "setosa").astype(int),
    (iris['Species'] == "versicolor").astype(int),
    (iris['Species'] == "virginica").astype(int)
))




Use the predict function and ensure that the predicted values are the same for both linear models: the one created with X  as its design matrix and the one created with Q as its design matrix.

In [ ]:
# Assign column names to the design matrix (no intercept)
col_names = ["setosa", "versicolor", "virginica"]

# Create DataFrames for x and Q


# Fit the OLS models with no intercept (since x and Q already have no intercept column)
mod3 = sm.OLS(y, df_x).fit()


# Use the predict method to get the fitted values
pred_mod3 = mod3.predict(df_x)


# Get the unique predicted values (rounded for numerical stability)


print("Unique predictions from mod3 (using x):", )
print("Unique predictions from mod4 (using Q):", )

Load the boston housing data and extract X and y. The dimensions are n = 506 and p = 13. Create a matrix that is (p + 1) x (p + 1) full of NA's. Label the columns the same columns as X. Do not label the rows. For the first row, find the OLS estimate of the y regressed on the first column only and put that in the first entry. For the second row, find the OLS estimates of the y regressed on the first and second columns of X only and put them in the first and second entries. For the third row, find the OLS estimates of the y regressed on the first, second and third columns of X only and put them in the first, second and third entries, etc. For the last row, fill it with the full OLS estimates.

In [ ]:
# Load the Boston housing data from the provided URL
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep=r"\s+", skiprows=22, header=None)

# The dataset is stored in two alternating rows.
# The first row of each pair contains predictor values,
# the second row contains the response (target) as well as extra predictors.
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

# Number of observations
n = data.shape[0]

# Construct the design matrix X with an intercept:
# First column is ones, then the 13 predictors.
X = np.hstack()
y = target

# Create column names: "Intercept" plus the 13 predictor names.
# (Using the standard Boston dataset names)
col_names = np.concatenate((["Intercept"], 
                            ["CRIM", "ZN", "INDUS", "CHAS", "NOX", 
                             "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT"]))

# Create a (p+1)x(p+1) matrix (14x14) filled with np.nan
M = np.full((X.shape[1], ), )

# For each i from 1 to 14, compute the OLS estimates using the first i columns of X.
for i in range(1, X.shape[1] + 1):
    b = np.full(X.shape[1], np.nan)  # Temporary vector of length 14 filled with np.nan
    X2 = X[:, :i]  # Use the first i columns of X
    # Compute OLS estimate: beta = (X2^T X2)^{-1} X2^T y
    beta = np.linalg.inv(X2.T @ X2) @ (X2.T @ y)
    b[:i] = beta  # Place the estimates in the first i entries
    M[i-1, :] = b

# Convert M to a DataFrame and label its columns
M_df = pd.DataFrame()
print(M_df)



Why are the estimates changing from row to row as you add in more predictors?

Create a vector of length p+1 and compute the R² values for each of the above models. 

In [ ]:
# Compute ybar and SST
ybar = 
SST =

# Initialize an array to hold the R² values for each model (there are 14 models)
Rsq = np.empty()

# Loop over i = 1 to 14 (each row of M corresponds to a model with the first i predictors)
p_plus_1 = M.shape[0]  # This should be 14
for i in range(1, p_plus_1 + 1):
    # Create a coefficient vector of length 14.
    # Use the first i coefficients from M (row i-1) and pad the rest with zeros.
    b = np.concatenate((M[i-1, :i], np.zeros(p_plus_1 - i)))
    
    # Compute the fitted values for the full design matrix X using these coefficients
    yhat = X @ b
    
    # Compute the regression sum of squares (SSR)
    SSR = 
    
    # Compute R² as SSR / SST
    Rsq[i-1] = 

print("R² values for each model:")
print(Rsq)

Is R² monotonically increasing? Why?

Create a 2x2 matrix with the first column 1's and the next column iid normals. Find the absolute value of the angle (in degrees, not radians) between the two columns in absolute difference from 90 degrees. 

In [ ]:
def norm_vec(v):
    return np.sqrt(np.sum(v**2))

# Create a 2x2 matrix: first column is all ones; second column is iid N(0,1)
X = np.ones()
X[:, 1] = np.

# Compute the cosine of the angle between the first and second column
cos_theta = np.dot(X[:, 0], X[:, 1]) / (norm_vec(X[:, 0]) * norm_vec(X[:, 1]))
print("Cosine of the angle:", cos_theta)

# Compute the angle in degrees
angle_degrees = np.arccos(cos_theta) * 180 / np.pi
print("Angle between columns (degrees):", angle_degrees)

# Compute the absolute difference from 90 degrees
abs_diff = 
print("Absolute difference from 90 degrees:", abs_diff)

Repeat this exercise `Nsim = 1e5` times and report the average absolute angle.

In [ ]:
Nsim = int(1e5)
angles = np.empty(Nsim)

for i in range(Nsim):
    # Construct a 2x2 matrix: first column is all ones, second column is two iid normals.
    X = np.ones((2, 2))
    X[:, 1] = np.random.randn(2)
    
    # Compute cosine of the angle between the two columns.
    cos_theta = np.dot(X[:, 0], X[:, 1]) / (norm_vec(X[:, 0]) * norm_vec(X[:, 1]))
    
    # Compute the angle in degrees.
    angle = np.arccos(cos_theta) * 180 / np.pi
    
    # Compute the absolute difference from 90 degrees.
    angles[i] = abs(90 - angle)

# Report the average absolute angle difference.
avg_angle = np.mean(angles)
print("Average absolute angle difference (degrees):", avg_angle)

Create a n x 2 matrix with the first column 1's and the next column iid normals. Find the absolute value of the angle (in degrees, not radians) between the two columns. For n = 10, 50, 100, 200, 500, 1000, report the average absolute angle over `Nsim = 1e5` simulations.

In [ ]:
def norm_vec(v):
    return np.linalg.norm(v)

# Define sample sizes (n values) as in R code
N_s = np.array()
Nsim = int(1e5)

# Create an empty array to store the angle differences
angles = np.empty((Nsim, len(N_s)))

# Loop over each sample size
for j, n_val in enumerate(N_s):
    for i in range(Nsim):
        # Create a n_val x 2 matrix: first column ones, second column iid normals.
      
        
        # Compute cosine of the angle between the two columns.
        # The first column is all ones, so its norm is sqrt(n_val).
       
        
        # Compute the angle in degrees
        
        
        # Compute the absolute difference from 90 degrees
     

# Compute the average absolute angle for each sample size
avg_angles = np.mean(angles, axis=0)
print("Average absolute angle differences (in degrees) for each n in N_s:")
print(avg_angles)

What is this absolute angle difference from 90 degrees converging to? Why does this make sense?

As the sample size grows, the constant column (all ones) becomes orthogonal to the random normal column (which averages to nearly 0), and so the angle between them becomes 90°, making the absolute difference from 90° shrink t